In [2]:
# Importing the required libraries
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error, median_absolute_error, explained_variance_score
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
Mutational Signature Dataset - Tally file

In [3]:
#############################
#### Signature Database #####
#############################
folder_path = '../../../Downloads/new_sigs/'
sig_data = pd.DataFrame()

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    # Filter to match the pattern 'SBS96_catalogue.<sample_name>.hg19.tally.csv' -  only doing sbs first
    if filename.startswith('SBS96_catalogue.') and filename.endswith('.hg19.tally.csv'):
        file_path = os.path.join(folder_path, filename)
        data = pd.read_csv(file_path, index_col=None)

        # Extract the sample name from the filename (e.g., 'TCGA-CA-6717-01')
        sample_name = filename.split('.')[1]
        select_data = data.drop(columns=['type', 'count'])

        # Pivot the DataFrame so that 'channel' becomes the columns
        pivot_data = select_data.pivot_table(index=None, columns="channel", values="fraction").reset_index(drop=True)
        pivot_data.insert(0, 'sample', sample_name)
        sig_data = pd.concat([sig_data, pivot_data], ignore_index=True)

# Add final column to data (empty) - maybe dont do it here??
sig_data["p53 status"] = "

# Display the final DataFrame or save it to a file
print("\n--------------------------\n")
print(sig_data)
print("\n--------------------------\n")


--------------------------

channel           sample   A[C>A]A   A[C>A]C   A[C>A]G  A[C>A]T   A[C>G]A  \
0        TCGA-A2-A0T5-01  0.000000  0.000000  0.000000  0.00000  0.000000   
1        TCGA-CF-A9FF-01  0.000000  0.010811  0.010811  0.00000  0.000000   
2        TCGA-CA-6717-01  0.000848  0.001774  0.000309  0.01049  0.000848   

channel   A[C>G]C   A[C>G]G  A[C>G]T   A[C>T]A  ...   T[T>A]T   T[T>C]A  \
0        0.000000  0.000000  0.00000  0.000000  ...  0.000000  0.000000   
1        0.000000  0.010811  0.00000  0.010811  ...  0.000000  0.005405   
2        0.000848  0.000000  0.00054  0.001465  ...  0.002854  0.006788   

channel   T[T>C]C   T[T>C]G   T[T>C]T   T[T>G]A   T[T>G]C  T[T>G]G   T[T>G]T  \
0        0.000000  0.000000  0.000000  0.000000  0.000000   0.0000  0.000000   
1        0.000000  0.000000  0.000000  0.000000  0.000000   0.0000  0.005405   
2        0.017123  0.009873  0.017663  0.007636  0.009873   0.0027  0.082298   

channel  p53 status  
0                 

MAF File: https://xenabrowser.net/datapages/?dataset=October_2016_whitelist_2583.snv_mnv_indel.maf.xena.nonUS&host=https%3A%2F%2Fpcawg.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443

In [83]:
#############################
######### MAF FILE ##########
#############################
maf_file_path = '../../../Downloads/October_2016_whitelist_2583.snv_mnv_indel.maf.xena.nonUS'
maf_data = pd.read_csv(maf_file_path, sep='\t', comment='#', low_memory=False)

filtered_data = maf_data[(maf_data['start'] == maf_data['end']) & (maf_data['alt'] != '-')]
filtered_data['HG19_Variant'] = 'chr17:g.' + \
                                (filtered_data['start'] - 1).astype(str) + \
                                filtered_data['reference'] + '>' + \
                                filtered_data['alt']

maf_filtered = filtered_data[['Sample', 'HG19_Variant', 'gene', 'effect']]
print(maf_filtered)

/var/folders/pr/sxpxd22s7jj41b9d9c6lk5yc0000gp/T/ipykernel_34917/2570192616.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['HG19_Variant'] = 'chr17:g.' + \


           Sample          HG19_Variant      gene             effect
0         DO46416    chr17:g.1230447G>A     ACAP3             Intron
1         DO46416    chr17:g.1609722C>T  SLC35E2B             Intron
2         DO46416    chr17:g.1903275C>T   Unknown                IGR
3         DO46416    chr17:g.2574998C>T     TTC34             Intron
4         DO46416    chr17:g.3151249G>A    PRDM16             Intron
...           ...                   ...       ...                ...
21980512  DO51503  chr17:g.150611902G>A   Unknown                IGR
21980513  DO51503  chr17:g.151231465T>C   Unknown                IGR
21980514  DO51503  chr17:g.151815421C>T     GABRQ  Missense_Mutation
21980515  DO51503  chr17:g.152000524T>C     NSDHL             Intron
21980516  DO51503  chr17:g.152538786G>A   Unknown                IGR

[20621575 rows x 4 columns]


Dataset: Download at https://p53.fr/download-the-database - Using .xlsx EU Variants

In [160]:
#############################
####### TP53 Database #######
#############################
new_file_tester =  '../../../Downloads/UMD_variants_EU.xlsx'
tester_data =  pd.read_excel(new_file_tester)

pattern = r'^chr17:g\.\d+[A, C, G, T]>[A, C, G, T]$'
tester_data_filtered = tester_data[tester_data['HG19_Variant'].str.contains(pattern, regex=True)]
tester_data_filtered['type'] = tester_data_filtered['Mutational_event']

/Users/asanghvi/Desktop/ML/ML_practice/.venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/var/folders/pr/sxpxd22s7jj41b9d9c6lk5yc0000gp/T/ipykernel_34917/465436135.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tester_data_filtered['type'] = tester_data_filtered['Mutational_event']


In [161]:
p53_db = tester_data_filtered[['HG19_Variant', 'Pathogenicity', 'Final comment']]

In [ ]:
#############################
###### Join Dataframes ######
#############################
merged_data = pd.merge(maf_filtered, p53_db[['HG19_Variant', 'Pathogenicity', 'Final comment']], 
                       on='HG19_Variant', how='left')
merged_data.fillna({ 'Pathogenicity' : 'Unknown' }, inplace=True)
merged_data.fillna({ 'Final comment' : 'No Comment' }, inplace=True)

pathogenicity_mapping = {
    'Pathogenic': 1,
    'Likely Pathogenic': 0.75,
    'Possibly pathogenic': 0.5,
    'VUS': 0.25,
    'Benign': 0,
    'Unknown': -1,
}

merged_data['p53 status'] = merged_data['Pathogenicity'].map(pathogenicity_mapping)
formatted_merged = merged_data[['Sample', 'HG19_Variant', 'gene', 'Pathogenicity', 'Final comment', 'p53 status']]
prepared_data = formatted_merged[(formatted_merged['p53 status'] != -1.0) & (formatted_merged['gene'] == 'TP53')]

In [186]:
print(merged_data)

           Sample          HG19_Variant      gene             effect  \
0         DO46416    chr17:g.1230447G>A     ACAP3             Intron   
1         DO46416    chr17:g.1609722C>T  SLC35E2B             Intron   
2         DO46416    chr17:g.1903275C>T   Unknown                IGR   
3         DO46416    chr17:g.2574998C>T     TTC34             Intron   
4         DO46416    chr17:g.3151249G>A    PRDM16             Intron   
...           ...                   ...       ...                ...   
20621570  DO51503  chr17:g.150611902G>A   Unknown                IGR   
20621571  DO51503  chr17:g.151231465T>C   Unknown                IGR   
20621572  DO51503  chr17:g.151815421C>T     GABRQ  Missense_Mutation   
20621573  DO51503  chr17:g.152000524T>C     NSDHL             Intron   
20621574  DO51503  chr17:g.152538786G>A   Unknown                IGR   

         Pathogenicity Final comment  p53 status  
0              Unknown    No Comment        -1.0  
1              Unknown    No Comm

In [198]:
def assign_pathogenicity_status(df):
    # Group by 'Sample' and determine pathogenicity for each group
    pathogenicity_status = df.groupby('Sample').apply(
        lambda group: 'Pathogenic' if ((group['gene'] == 'TP53') & 
                                       (group['Pathogenicity'].isin(['Pathogenic', 'Likely Pathogenic']))).any() 
                      else 'Benign'
    ).reset_index(name='Pathogenicity')
    
    return pathogenicity_status

new_merged_data = assign_pathogenicity_status(merged_data)

print(new_merged_data)

      Sample Pathogenicity
0     DO1000        Benign
1     DO1001    Pathogenic
2     DO1002        Benign
3     DO1003        Benign
4     DO1004        Benign
...      ...           ...
1777  DO7196        Benign
1778  DO7214        Benign
1779  DO7280        Benign
1780  DO7304        Benign
1781  DO7328        Benign

[1782 rows x 2 columns]


/var/folders/pr/sxpxd22s7jj41b9d9c6lk5yc0000gp/T/ipykernel_34917/4119386144.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pathogenicity_status = df.groupby('Sample').apply(


In [199]:
def assign_pathogenicity_status(df):
    # Group by 'Sample' and assign p53 status as a numeric value (1 for pathogenic, 0 for benign)
    pathogenicity_status = df.groupby('Sample').apply(
        lambda group: 1 if ((group['gene'] == 'TP53') & 
                            (group['p53 status'].astype(float) > 0.5)).any() 
                      else 0
    ).reset_index(name='p53 status')
    
    return pathogenicity_status

# Run the function and print the result
new_merged_data_temp = assign_pathogenicity_status(merged_data)
print(new_merged_data_temp)

      Sample  p53 status
0     DO1000           0
1     DO1001           1
2     DO1002           0
3     DO1003           0
4     DO1004           0
...      ...         ...
1777  DO7196           0
1778  DO7214           0
1779  DO7280           0
1780  DO7304           0
1781  DO7328           0

[1782 rows x 2 columns]


/var/folders/pr/sxpxd22s7jj41b9d9c6lk5yc0000gp/T/ipykernel_34917/2780195078.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pathogenicity_status = df.groupby('Sample').apply(


In [207]:
##############################
###### Machine Learning ######
##############################
X = new_merged_data_temp.drop(columns=['Sample'])
y = new_merged_data_temp['p53 status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [208]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [209]:
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       345
           1       1.00      1.00      1.00        12

    accuracy                           1.00       357
   macro avg       1.00      1.00      1.00       357
weighted avg       1.00      1.00      1.00       357



In [151]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score 

print(sig_data)

X_sig = sig_data.drop(columns=['p53 status'])  # Drop target column in sig_data
y = sig_data['p53 status']                     # Target label (p53 status)

X_channel = prepared_data.drop(columns=['gene', 'Final comment'])
print("\n--------------------------\n")
print(X_channel)

numerical_features = sig_data.columns[2:-1]
sig_preprocessor = ColumnTransformer([
    ('scaler', StandardScaler(), numerical_features),
    ('encoder', OneHotEncoder(), ['sample'])
])

channel_preprocessor = ColumnTransformer([
    ('scaler', StandardScaler(), ['p53 status']),
    ('encoder', OneHotEncoder(), ['channel', 'HG19_Variant', 'Pathogenicity'])
])

X_sig_transformed = sig_preprocessor.fit_transform(X_sig)
X_channel_transformed = channel_preprocessor.fit_transform(X_channel)

print("\n--------------------------\n")
print(X_sig_transformed)
print("\n--------------------------\n")
print(X_channel_transformed)

channel           sample   A[C>A]A   A[C>A]C   A[C>A]G  A[C>A]T   A[C>G]A  \
0        TCGA-A2-A0T5-01  0.000000  0.000000  0.000000  0.00000  0.000000   
1        TCGA-CF-A9FF-01  0.000000  0.010811  0.010811  0.00000  0.000000   
2        TCGA-CA-6717-01  0.000848  0.001774  0.000309  0.01049  0.000848   

channel   A[C>G]C   A[C>G]G  A[C>G]T   A[C>T]A  ...   T[T>A]T   T[T>C]A  \
0        0.000000  0.000000  0.00000  0.000000  ...  0.000000  0.000000   
1        0.000000  0.010811  0.00000  0.010811  ...  0.000000  0.005405   
2        0.000848  0.000000  0.00054  0.001465  ...  0.002854  0.006788   

channel   T[T>C]C   T[T>C]G   T[T>C]T   T[T>G]A   T[T>G]C  T[T>G]G   T[T>G]T  \
0        0.000000  0.000000  0.000000  0.000000  0.000000   0.0000  0.000000   
1        0.000000  0.000000  0.000000  0.000000  0.000000   0.0000  0.005405   
2        0.017123  0.009873  0.017663  0.007636  0.009873   0.0027  0.082298   

channel  p53 status  
0                    
1                    
2  

In [138]:
# # prepare input data
# from sklearn.preprocessing import OneHotEncoder

# # prepare input data
# def prepare_inputs(X_train, X_test):
# 	ohe = OneHotEncoder()
# 	ohe.fit(X_train)
# 	X_train_enc = ohe.transform(X_train)
# 	X_test_enc = ohe.transform(X_test)
# 	return X_train_enc, X_test_enc

# # prepare target
# def prepare_targets(y_train, y_test):
# 	le = LabelEncoder()
# 	le.fit(y_train)
# 	y_train_enc = le.transform(y_train)
# 	y_test_enc = le.transform(y_test)
# 	return y_train_enc, y_test_enc

# # encoding not working?


# # prepare input data
# X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# # prepare output data
# # y_train_enc, y_test_enc = prepare_targets(y_train, y_test)